In [1]:
import numpy as np
import pandas as pd
import json
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pickle as pkl
import imblearn
np.random.seed(1332)
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import sys                                               #### Filling the missing values using KNN because the dataset is very less
from impyute.imputation.cs import fast_knn
import imblearn                                      ### sampling the data using the smote method
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
import sklearn
import xgboost
from xgboost import XGBClassifier              ### XGBoost classification method
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from xgboost import cv

In [2]:
excel = pd.ExcelFile('Coronna Data CERTAIN with KVB edits.xlsx')
df = pd.read_excel(excel, 'BL+3M')

In [3]:
df = df.drop(columns = ['SubjectID', 'CDate', 'Match'])    ### Unique values
df = df.drop(columns = ['SubjectID.1', 'UNMC_id.1', 'CDate.1', 'init_group.1', 'grp.1', 'UNMC_id.2', 'grp.2', 'init_group.2', 'CDate.1', 'futime.1'])    ### Duplicate values
df = df.drop(columns = ['init_group', 'futime'])    ### same values for all rows

In [4]:
df = df.drop(columns = ['statin_use', 'rfstatus_impute', 'ccpstatus_impute', 'statin_use.1', 'smkyrs', 'numcigs'])  ### Dropping columns who have null values greater than 70%

In [5]:
df = df.drop(columns = ['3MResponse','DAS28-CRP BL'])

In [6]:
df = df.rename(columns={"usresultsIgA.1": "usresultsIgA_BL", "usresultsIgG.1": "usresultsIgG_BL", "usresultsIgM.1": "usresultsIgM_BL", 'seatedbp1.1': 'seatedbp1_BL',
                              'seatedbp2.1': 'seatedbp2_BL', 'pres_mtx.1': 'pres_mtx_BL', 'pres_arava.1': 'pres_arava_BL', 'pres_azulfidine.1': 'pres_azulfidine_BL', 
                              'pres_plaquenil.1': 'pres_plaquenil_BL', 'pres_imuran.1': 'pres_imuran_BL', 'pres_minocin.1': 'pres_minocin_BL', 'pres_pred.1': 'pres_pred_BL',
                              'statin_use.1': 'statin_use_BL', 'tender_jts_28.1': 'tender_jts_28_BL', 'BLswollen_jts_28': 'swollen_jts_28_BL',
                              'BLmd_global_assess': 'md_global_assess_BL', 'BLpt_global_assess': 'pt_global_assess_BL', 'BLdi': 'di_BL', 'BLpt_pain': 'pt_pain_BL', 'BLusresultsCRP': 'usresultsCRP_BL',
                              'DAS28-CRP BL': 'DAS28-CRP_BL', })     ### Renaming some columns. Not sure why - did it because previous student did it

In [7]:
df = df.drop('UNMC_id',1)
final_df = df
model_label = LabelEncoder()
final_df['grp'] = model_label.fit_transform(final_df['grp'].astype('str'))
final_df['gender'] = model_label.fit_transform(final_df['gender'].astype('str'))
final_df['final_education'] = model_label.fit_transform(final_df['final_education'].astype('str'))
final_df['race_grp'] = model_label.fit_transform(final_df['race_grp'].astype('str'))
final_df['newsmoker'] = model_label.fit_transform(final_df['newsmoker'].astype('str'))
final_df['drinker'] = model_label.fit_transform(final_df['drinker'].astype('str'))
final_df['ara_func_class'] = model_label.fit_transform(final_df['ara_func_class'].astype('str'))

<ipython-input-7-f80c6ef2a649>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('UNMC_id',1)


In [8]:
#final_df = final_df[final_df['DAS28-CRP 3m'] != 'Unknown']    ### We don't need unknown category in our classification data so removing that category
final_df = final_df[final_df['DAS28-CRP 3m'].notnull()]

In [22]:
rs = 1337
X = final_df.drop('DAS28-CRP 3m',1)                            ### Dividing the dataframe into X and y set
y = final_df['DAS28-CRP 3m']
y = model_label.fit_transform(y.astype('str'))
X_train_norm, X_val_norm, Y_train, Y_val = train_test_split(X, y, test_size=0.1,random_state = rs)

<ipython-input-22-8d086750a9c9>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = final_df.drop('DAS28-CRP 3m',1)                            ### Dividing the dataframe into X and y set


In [10]:
#X_train_norm['y'] = Y_train
#X_val_norm['y'] = Y_val

In [23]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=15)
fit_train = imputer.fit(X_train_norm)
imput_train = fit_train.transform(X_train_norm)
c = list(X_train_norm.columns)
imput_train_df = pd.DataFrame(imput_train, columns = c)

In [24]:
imput_test = fit_train.transform(X_val_norm)
c = list(X_val_norm.columns)
imput_test_df = pd.DataFrame(imput_test, columns = c)

In [25]:
labels = ['grp','gender','final_education','race_grp','newsmoker','drinker','ara_func_class']     ### normalizing the dataset

for i in imput_train_df.columns:
    if i not in labels:
        mean = imput_train_df[i].mean()
        std = imput_train_df[i].std()
        imput_train_df[i] = (imput_train_df[i] - mean) / std
        imput_test_df[i] = (imput_test_df[i] - mean) / std

In [26]:
labels = ['pres_imuran','pres_minocin','num_tnf','num_nontnf','hxunstab_ang','pres_minocin_BL','ethnicity','hxstroke','pres_imuran_BL']    #### This columns has same value for every row so when I divide by mean and std the value goes to infinity
for i in labels:
    imput_train_df = imput_train_df.drop(i,1)
    imput_test_df = imput_test_df.drop(i,1)

<ipython-input-26-649ae34e813e>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  imput_train_df = imput_train_df.drop(i,1)
<ipython-input-26-649ae34e813e>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  imput_test_df = imput_test_df.drop(i,1)


In [27]:
# def sampling(over_under,X,y):
#     if over_under == 'under':
#         sampler = imblearn.under_sampling.RandomUnderSampler()
#         X_under, y_under = sampler.fit_resample(X, y)
#         return X_under,y_under
#     elif over_under == 'over':
#         ros = RandomOverSampler(random_state=rs)
#         X_over, y_over = ros.fit_resample(X, y)
#         return X_over,y_over
#     else:
#         sampler = SMOTE(random_state = rs)
#         X_smote, y_smote = sampler.fit_resample(X, y)
#         return X_smote,y_smote
# X_final,y_final = sampling('smote',imput_train_df,Y_train)

In [28]:
imput_train_df['y'] = Y_train
imput_test_df['y'] = Y_val

In [29]:
from autogluon.tabular import TabularDataset, TabularPredictor
predictor = TabularPredictor(label='y', eval_metric = 'r2').fit(imput_train_df)

No path specified. Models will be saved in: "AutogluonModels/ag-20220405_161759\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220405_161759\"
AutoGluon Version:  0.4.0
Python Version:     3.8.5
Operating System:   Windows
Train Data Rows:    250
Train Data Columns: 68
Label Column: y
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (277, 0, 142.336, 80.44499)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    7818.74 MB
	Train Data (Original)  Memory Usage: 0.14 MB (0.0% of available memory)
	Inferring data type of each 

In [30]:
predictor.evaluate(imput_test_df)

Evaluation: r2 on test data: 0.9637536615980647
Evaluations on test data:
{
    "r2": 0.9637536615980647,
    "root_mean_squared_error": -13.581291072465865,
    "mean_squared_error": -184.451467195041,
    "mean_absolute_error": -11.839905670710973,
    "pearsonr": 0.9867651792010529,
    "median_absolute_error": -11.967399597167969
}


{'r2': 0.9637536615980647,
 'root_mean_squared_error': -13.581291072465865,
 'mean_squared_error': -184.451467195041,
 'mean_absolute_error': -11.839905670710973,
 'pearsonr': 0.9867651792010529,
 'median_absolute_error': -11.967399597167969}

In [16]:
from autogluon.tabular import TabularDataset, TabularPredictor

save_path = 'model-weights'
predictor = TabularPredictor(label='y', path=save_path).fit(X_final)

Beginning AutoGluon training ...
AutoGluon will save models to "model-weights\"
AutoGluon Version:  0.4.0
Python Version:     3.8.5
Operating System:   Windows
Train Data Rows:    279
Train Data Columns: 68
Label Column: y
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	3 unique label values:  [2, 1, 0]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 3
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    5202.66 MB
	Train Data (Original)  Memory Usage: 0.15 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.


In [ ]:
#test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
#y_test = test_data[label]  # values to predict
#test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating

In [ ]:
type(Y_val)

In [17]:
Y_val = pd.Series(Y_val)

In [ ]:
type(Y_val)


In [ ]:
print(predictor)

In [18]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(X_val_norm)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=Y_val, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.25
Evaluations on test data:
{
    "accuracy": 0.25,
    "balanced_accuracy": 0.3636363636363637,
    "mcc": -0.029821603968282907
}


Predictions:  
 8      1
195    1
174    2
72     2
242    2
118    2
206    2
196    1
288    2
295    2
198    1
106    2
129    2
213    1
3      2
175    2
13     1
69     2
202    2
18     2
163    2
23     1
265    2
287    1
87     2
56     2
149    2
78     2
Name: y, dtype: int32


In [ ]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

In [ ]:
label = 'class'
print("Summary of class variable: \n", train_data[label].describe())

In [ ]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

In [ ]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
print(type(y_test), y_test)
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

In [ ]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

In [11]:
#import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
#excel = pd.ExcelFile('Coronna Data CERTAIN with KVB edits.xlsx')
#train_data = pd.read_excel(excel, 'BL+3M')
predictor = TabularPredictor(label='y').fit(X_train_norm)


No path specified. Models will be saved in: "AutogluonModels/ag-20220405_154934\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220405_154934\"
AutoGluon Version:  0.4.0
Python Version:     3.8.5
Operating System:   Windows
Train Data Rows:    250
Train Data Columns: 77
Label Column: y
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	3 unique label values:  [2, 1, 0]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 3
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    8610.29 MB
	Train Data (Original)  Memory Usage: 0.15 MB (0.0% of available memory)
	Inferring data type of each featu

In [12]:
predictor.evaluate(X_val_norm)

Evaluation: accuracy on test data: 0.8571428571428571
Evaluations on test data:
{
    "accuracy": 0.8571428571428571,
    "balanced_accuracy": 0.8535353535353535,
    "mcc": 0.7848854747681295
}


{'accuracy': 0.8571428571428571,
 'balanced_accuracy': 0.8535353535353535,
 'mcc': 0.7848854747681295}